<a href="https://colab.research.google.com/github/seraogianluca/mircv-exercises/blob/main/Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#To support GPU
!pip install opencv-python==4.4.0.46

In [ ]:
import os
import cv2
import numpy as np

# for showing images in the cell outputs (Jupyter Notebooks / Google Colab)
from IPython.display import display
from ipywidgets import Image

BASE_DIR = '/content/gdrive/My Drive/mircv2021'
DEEP_PROTO = BASE_DIR + '/data/caffe/train_val.prototxt'
DEEP_MODEL = BASE_DIR + '/data/caffe/bvlc_reference_caffenet.caffemodel'
SRC_FOLDER = BASE_DIR + '/data/coco_img'

DEEP_LAYER = 'relu7'
SIZE = (227, 227)
MEAN_VALUES = (104, 117, 123)  # BGR

K = 10

In [ ]:
# Image displaying
filename = "000000000724.jpg"
image1 = os.path.join(SRC_FOLDER, filename)
display(Image.from_file(image1,width=300,height=400))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

In [ ]:
# feature extraction
img1 = cv2.imread(image1)
net = cv2.dnn.readNetFromCaffe(DEEP_PROTO, DEEP_MODEL)
blob1 = cv2.dnn.blobFromImage(img1, 1.0, SIZE, MEAN_VALUES, swapRB=False, crop=False)
net.setInput(blob1)
prob1 = net.forward(DEEP_LAYER)
print(prob1)
print(prob1.shape)

prob1 = prob1.flatten()
print(prob1)
print(prob1.shape)

[[-0.         -0.         -0.         ...  0.98092026 -0.
  -0.        ]]
(1, 4096)
[-0.         -0.         -0.         ...  0.98092026 -0.
 -0.        ]
(4096,)


In [ ]:
# normalization

prob1 = prob1/np.linalg.norm(prob1)
print(prob1)

[-0.         -0.         -0.         ...  0.01489696 -0.
 -0.        ]


In [ ]:
# self-similarity

s = np.dot(prob1,prob1)
print(s)

0.9999999


In [ ]:
# similarity

filename = "000000001268.jpg"

image2 = os.path.join(SRC_FOLDER, filename)
display(Image.from_file(image2,width=300,height=400))

img2 = cv2.imread(image2)

blob2 = cv2.dnn.blobFromImage(img2, 1.0, SIZE, MEAN_VALUES, swapRB=False, crop=False)
net.setInput(blob2)
prob2 = net.forward(DEEP_LAYER)
prob2 = prob2.flatten()
prob2 = prob2/np.linalg.norm(prob2)

s = np.dot(prob1,prob2)
print(s)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x0cXICC_PROFILE\x00\x01\x…

0.24600789


In [ ]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values
        #TODO
        #Load Caffe Weights
        self.net = cv2.dnn.readNetFromCaffe(net_proto_path, trained_model_path)
        
        # to enable GPU (this won't work on Colab without recompiling opencv)
        #self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        #self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    def extract(self, img_file, layer, normalize=False):
        #TODO
        #Load img_file and extract fetures calling DNN
        img = cv2.imread(img_file)
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)

        features = self.net.forward(layer).flatten()

        #normalize fetaures if normalize = True
        if normalize == True:
          features = features/np.linalg.norm(features)

        #return the features
        return features
    
# it creates an instance of the DNNExtractor class
dnn = DNNExtractor(DEEP_PROTO, DEEP_MODEL, SIZE, MEAN_VALUES)

In [ ]:
def extract_features(img_folder):
  
    #TODO
    #get the list of image filenames
    filenames = os.listdir(img_folder)

    descriptors = []
    #TODO
    #cycle the img_folder directory to get the image files
    analyzed = 0
    for count, img in enumerate(filenames[:100]):
      #extract the image features
      filepath = os.path.join(img_folder, img)
      features = dnn.extract(filepath, DEEP_LAYER, normalize=True)

      #add the features to the descriptors list
      descriptors.append(features)

      analyzed += count
    
    print("Analyzed ", analyzed, " images.")

    #convert descriptors and filenames as Numpy arrays
    descriptors = np.array(descriptors)
    filenames = np.array(filenames)

    return descriptors, filenames

descs, ids = extract_features(SRC_FOLDER)

Analyzed  4950  images.


In [ ]:
def search(queryF, k):
    #TODO
    #evaluate the dot products between descs and queryF
    dots = np.dot(descs, queryF)
    
    #To sort the results create a zip between dot products and ids
    results = zip(dots, ids)

    #then call the sorted function to sort them
    results = sorted(results, reverse=True)
    #return the k best results
    return results[:k]

  
def display_results(results):
    for score, filename in results:
        print('{} - {:.3f}'.format(filename, score))  # :.3f = 3-decimal float

        filepath = os.path.join(SRC_FOLDER, filename)
        image = Image.from_file(filepath, width=300, height=400)
        display(image)

img_query = os.path.join(SRC_FOLDER, "000000321557.jpg")
query_features = dnn.extract(img_query, DEEP_LAYER, True)

res = search(query_features, K)
print(res)
display_results(res)

[(0.52089345, '000000297147.jpg'), (0.50620055, '000000408830.jpg'), (0.33964795, '000000102331.jpg'), (0.29106942, '000000343934.jpg'), (0.26391506, '000000388258.jpg'), (0.25614512, '000000472623.jpg'), (0.24356474, '000000143556.jpg'), (0.22365928, '000000305343.jpg'), (0.20455357, '000000176857.jpg'), (0.20180032, '000000227478.jpg')]
000000297147.jpg - 0.521


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000408830.jpg - 0.506


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000102331.jpg - 0.340


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe2\x02@ICC_PROFILE\x00\x01\x…

000000176857.jpg - 0.205


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000227478.jpg - 0.202


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

In [ ]:
#Optional task

#TODO
# binary descriptors. For each value set 1 if value > 0, 0 otherwise
descs_bin = np.where(descs == 0, 0, 1)
print(descs_bin)

def search_bin(queryF_bin, k):
    #TODO
    #evaluate Hamming distance between descs_bin and queryF_bin
    hamming = []
    for desc in descs_bin:
      ham = np.sum(np.bitwise_xor(desc,queryF_bin))
      hamming.append(ham)
    
    #then call the sorted function to sort them
    results = zip(hamming, ids)
    results = sorted(results, reverse=False)

    #return the k best results
    return results[:k]

[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [ ]:
#Optional task

#TODO
# binary query. For each value set 1 if value > 0, 0 otherwise
queryF_bin = np.where(query_features == 0, 0, 1)

res = search_bin(queryF_bin, K)
print(res)
display_results(res)


[(638, '000000408830.jpg'), (682, '000000297147.jpg'), (764, '000000102331.jpg'), (836, '000000198805.jpg'), (883, '000000143556.jpg'), (885, '000000306893.jpg'), (906, '000000467776.jpg'), (920, '000000305343.jpg'), (960, '000000388258.jpg'), (965, '000000192871.jpg')]
000000408830.jpg - 638.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000297147.jpg - 682.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\…

000000102331.jpg - 764.000


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xe2\x02@ICC_PROFILE\x00\x01\x…

000000388258.jpg - 960.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x10\xecICC_PROFILE\x00\x0…

000000192871.jpg - 965.000


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x05XICC_PROFILE\x00\x01\x…